In [3]:
import unicodedata

from bs4 import BeautifulSoup
import sqlite3
import requests
import re
import json
import pandas as pd

class scrapper:
    @staticmethod
    def get_arks(year):
        url_base = "https://gallica.bnf.fr/ark:/12148/cb34363188x/date"+str(year)+"0101"
        req = requests.get(url_base, headers={'User-Agent': 'Mozilla/5.0'})
        p = req.content
        L = re.findall(r"https://gallica.bnf.fr/ark:/(\d*/(bpt\w*))?",str(p))
        S = set()
        for i in L:
            if(i[1] != ""):
                S.add(i[1])
        return S
    @staticmethod
    def get_pagination(ark):
        """fonction qui renvoie le nombre de pages d'un documents"""
        #print(cpt,"--> https://gallica.bnf.fr/"+i+"/f7/highres")
        req = requests.get("https://gallica.bnf.fr/services/Pagination?ark=" + ark)
        p = req.content
        pages = re.search(r"<nbVueImages>(\d*)</nbVueImages>", str(p))
        return pages.groups()[0]
    @staticmethod
    def get_page(ark, page, number = 1):
        
        req = requests.get(f"https://gallica.bnf.fr/ark:/12148/{ark}/f{page}n{number}.texteBrut")
        return req.content
    @staticmethod
    def get_document(ark, mode = "texteBrut"):
        req = requests.get("https://gallica.bnf.fr/ark:/12148/"+ark+"."+ mode)
        return req.content
    

In [9]:
class Cleaner:
    def __init__(self, directory):
        self.directory = directory
        pass
    def extract(self, file):
        soup = BeautifulSoup(file , "html.parser")
        df = pd.DataFrame(columns=["page", "arrêt", "date", "juridiction"])
        Decision, notes, page, new_page = False, False, 0, True
        for tag in soup.body : 
            if tag.name == "hr":
                page += 1
                notes = False
                new_page  = True
            if tag.name == "p" and tag.string is not None and not new_page:
                m1 = re.match(r".*?LA COUR(.+)", tag.string)
                m2 = re.match(r"(.+?)Du(.+?\d{3}.+?)(—|-)(.+?)(—|,).+", tag.string ) 
                if not Decision and m1:
                    Decision = True
                    text = m1.groups()[0]
                    First_page = page
                elif Decision and m2 :
                    Decision = False
                    text += m2.groups()[0]
                    date = m2.groups()[1]
                    juridiction = m2.groups()[3]
                    df = df.append({'page' : First_page, 'arrêt' : text , "date": date, "juridiction" : juridiction},ignore_index=True)
                    text = ''
                elif not notes and Decision:
                    if not re.match(r"^\(\d*\).+$", tag.string):
                        text+= tag.string + "\n"
                    else:
                        notes = True
                else:
                    pass
            else :
                new_page = False
        return df
    def save(self, df, ark):
        df.to_csv(f"{self.directory}/{ark}.csv", encoding="utf-8")
        pass
        

In [5]:
years = scrapper.get_arks(1830)
years

{'bpt6k5745965s', 'bpt6k57460598'}

In [10]:
file = scrapper.get_page("bpt6k5745965s", 179)

In [6]:
c = Cleaner("test")
file = scrapper.get_document("bpt6k5745965s")


In [11]:
c = Cleaner("test")
df = c.extract(file)
df

,page,arrêt,date,juridiction
0,1,"; — Attendu , 1 °. que ces expressions de l'a...",12 mai 1830.,Ch. civ.


In [14]:
df.loc[0]

page                                                           1
arrêt           ; — Attendu , 1 °. que ces expressions de l'a...
date                                                12 mai 1830.
juridiction                                            Ch. civ. 
Name: 0, dtype: object

In [8]:
def clean(file,ark, index_directories= "test"):
    """function to clean with a regex : a bit faster but with erros on the pages """
    soup = BeautifulSoup(file , "html.parser")
    count = 1
    df = pd.DataFrame(columns=["page", "arrêt", "date", "juridiction"])
    text = soup.text
    complete = re.findall(r"LA COUR(;|:).+?([A-Za-z].+?)\. Du(.+?(\d{3}|\d{2}.\d).+?)\.?\s?(—|-)(.+?)\.?(—|,)", text)
    end = re.findall(r"(.+)Du(.+?(\d{3}|\d{2}.\d).+?)\.?\s?(—|-)(.+?)\.?(—|,)")
    for _, arret, date, _,_,region,_ in m:
        df = df.append({'page' : page, 'arrêt' : arret , "date": date, "juridiction" : region},ignore_index=True)
    return df

In [197]:
clean(scrapper.get_page("bpt6k5745965s", )